In [1]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json

In [2]:
def g(string):

    string = string.replace("\xa0", " ")
    string = string.replace("&nbsp;", " ")
    string = string.replace("&thinsp;", "")
    string = string.replace("\u2009", "")
    return string

In [3]:
def get_links(text):
    ua = fake_useragent.UserAgent()
    data = requests.get(
        url=f"https://hh.ru/search/resume?text={text}&area=113&isDefaultArea=true&ored_clusters=true&order_by=relevance&search_period=0&logic=normal&pos=full_text&exp_period=all_time&page=1",
        headers={"user-agent":ua.random}
    )
    if data.status_code != 200:
        return
    soup = BeautifulSoup(data.content, "lxml")
    try:
        page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span",recursive=False)[-1].find("a").find("span").text)
    except:
        return
    for page in range(page_count):
        try:
            data = requests.get(
                url=f"https://hh.ru/search/resume?text={text}&area=113&isDefaultArea=true&ored_clusters=true&order_by=relevance&search_period=0&logic=normal&pos=full_text&exp_period=all_time&page={page}",
                headers={"user-agent":ua.random}
            )
            if data.status_code != 200:
                continue
            soup = BeautifulSoup(data.content, "lxml")
            for a in soup.find_all("a", attrs={"class":"bloko-link"}):
                yield f"https://hh.ru{a.attrs['href'].split('?')[0]}"
        except Exception as e:
            print(f"{e}")
        time.sleep(1)

In [4]:
def get_resume(link):
    ua = fake_useragent.UserAgent()
    data = requests.get(
        url = link,
        headers={"user-agent":ua.random}
    )
    if data.status_code != 200:
        return
    soup = BeautifulSoup(data.content, "lxml")
    try:
        name = g(soup.find(attrs={"class":"resume-block__title-text"}).text)
    except:
        name = ""

    try:
        salary = g(soup.find(attrs={"class":"resume-block__salary"}).text)
    except:
        salary = ""
    
    try:
        age = g(soup.find(attrs={"data-qa":"resume-personal-age"}).text)
    except:
        age = ""

    try:
        male = g(soup.find(attrs={"data-qa":"resume-personal-gender"}).text)
    except:
        male = ""

    try:
        job_s = g(soup.find(attrs={"class":"resume-job-search-status"}).text)
    except:
        job_s = ""

    try:
        expir = g(soup.find(attrs={"class":"resume-block__title-text resume-block__title-text_sub"}).text.replace("Опыт работы ", ""))
    except:
        expir = ""

    try:
        skills = [tag.text for tag in soup.find(attrs={"class":"bloko-tag-list"}).find_all(attrs={"class":"bloko-tag__section bloko-tag__section_text"})]
    except:
        skills = []  
    resume = {
        "ИМЯ": name,
        "ПОЛ": male,
        "ВОЗРАСТ": age,
        "ПОИСК": job_s,
        "ЗАРПЛАТА": salary,
        "НАВЫКИ": skills,
        "ОПЫТ": expir,
        "ССЫЛКА": link

    }

    return resume

In [5]:
if __name__ == "__main__":
    for a in get_links("учитель"):
        print(get_resume(a))
        time.sleep(1)

{'ИМЯ': '', 'ПОЛ': '', 'ВОЗРАСТ': '', 'ПОИСК': '', 'ЗАРПЛАТА': '', 'НАВЫКИ': ['Москва'], 'ОПЫТ': '', 'ССЫЛКА': 'https://hh.ru/search/resume/advanced'}
{'ИМЯ': '', 'ПОЛ': '', 'ВОЗРАСТ': '', 'ПОИСК': '', 'ЗАРПЛАТА': '', 'НАВЫКИ': ['Москва'], 'ОПЫТ': '', 'ССЫЛКА': 'https://hh.ru/search/resume/advanced'}
{'ИМЯ': '', 'ПОЛ': '', 'ВОЗРАСТ': '', 'ПОИСК': '', 'ЗАРПЛАТА': '', 'НАВЫКИ': [], 'ОПЫТ': '', 'ССЫЛКА': 'https://hh.ru/search/resume'}
{'ИМЯ': '', 'ПОЛ': '', 'ВОЗРАСТ': '', 'ПОИСК': '', 'ЗАРПЛАТА': '', 'НАВЫКИ': [], 'ОПЫТ': '', 'ССЫЛКА': 'https://hh.ru/article/23786'}
{'ИМЯ': 'Учитель физики', 'ПОЛ': 'Женщина', 'ВОЗРАСТ': '78 лет', 'ПОИСК': '', 'ЗАРПЛАТА': '', 'НАВЫКИ': ['Русский — Родной'], 'ОПЫТ': '56 лет 6 месяцев', 'ССЫЛКА': 'https://hh.ru/resume/f4d9c1670002bbd0280039ed1f653857776c57'}
{'ИМЯ': 'Учитель математики', 'ПОЛ': 'Мужчина', 'ВОЗРАСТ': '61 год', 'ПОИСК': '', 'ЗАРПЛАТА': '30000 ₽ на руки', 'НАВЫКИ': ['учитель'], 'ОПЫТ': '12 лет 10 месяцев', 'ССЫЛКА': 'https://hh.ru/resume/3ec439

KeyboardInterrupt: 